In [1]:
import os
import sys
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
import nltk
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn import svm
from sklearn.model_selection import cross_val_score
import string
from string import punctuation
import collections
from collections import Counter
from itertools import chain
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import KFold
import sklearn.model_selection as model_selection
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
import warnings
warnings.filterwarnings('ignore')

In [2]:
# importing the file with the coded tweets
df_1_new = pd.read_csv('scores_df1_full.csv')

# new dataframe with relevant columns
df1 = df_1_new[['Date', 'clean_no_stops', 'sentiment code', 'category']].copy()

# subset with manually coded tweets
subset_1 = df1[0:1235]
subset_1.dropna(inplace=True)
subset_1.reset_index(inplace=True)

In [3]:
# split words into lists
v = subset_1['clean_no_stops'].str.split().tolist()
# compute global word frequency
c = Counter(chain.from_iterable(v))
# filter, join, and re-assign
subset_1['clean_new'] = [' '.join([j for j in i if c[j] > 3]) for i in v] # remove low frequency words

subset_1["count"] = ""
for i in range(len(subset_1)):
    subset_1['count'][i] = int(str(len(subset_1['clean_new'][i].split())))

subset_1 = subset_1[subset_1['count'] > 5] # keep tweets with more than 5 words

In [4]:
# manual stemming to dominant words in the dataset
subset_1['clean_new'] = (subset_1['clean_new'].str.replace('השחקן','שחקן').str.replace('השחקנים','שחקן')
                            .str.replace('כשחקן','שחקן').str.replace('שחקני','שחקן').str.replace('לשחקן','שחקן')
                            .str.replace('לשחקני','שחקן').str.replace('בשחקן','שחקן').str.replace('מהשחקנים','שחקן')
                            .str.replace('ושחקנים','שחקן').str.replace('והשחקן','שחקן').str.replace('ישחק','שחקן'))

subset_1['clean_new'] = (subset_1['clean_new'].str.replace('ערבים','ערבי').str.replace('הערבי','ערבי')
                            .str.replace('הערבים','ערבי').str.replace('לערבים','ערבי').str.replace('והערבים','ערבי')
                            .str.replace('שהערבים','ערבי').str.replace('בערבית','ערבי').str.replace('ערביי','ערבי')
                            .str.replace('ערבית','ערבי').str.replace('וערבים','ערבי'))


subset_1['clean_new'] = (subset_1['clean_new'].str.replace('במשחק','משחק').str.replace('המשחק','משחק')
                            .str.replace('למשחק','משחק').str.replace('ומשחקים','משחק').str.replace('שמשחק','משחק')
                            .str.replace('המשחקים','משחק').str.replace('ומשחק','משחק').str.replace('מהמשחק','משחק')
                            .str.replace('שמשחקים','משחק').str.replace('משחקי','משחק').str.replace('למשחקים','משחק')
                           .str.replace('משחקת','משחק').str.replace('ומשחקת','משחק').str.replace('במשחקים','משחק'))

subset_1['clean_new'] = (subset_1['clean_new'].str.replace('בנבחרת','נבחרת').str.replace('הנבחרת','נבחרת')
                            .str.replace('לנבחרת','נבחרת').str.replace('מהנבחרת','נבחרת').str.replace('שהנבחרת','נבחרת')
                            .str.replace('כשהנבחרת','נבחרת').str.replace('בנבחרות','נבחרת'))


subset_1['clean_new'] = (subset_1['clean_new'].str.replace('ישראלי','ישראל').str.replace('בישראל','ישראל')
                            .str.replace('הישראלי','ישראל').str.replace('לישראל','ישראל').str.replace('ישראלים','ישראל')
                            .str.replace('ישראלית','ישראל').str.replace('והישראלים','ישראל').str.replace('מהישראלים','ישראל')
                           .str.replace('הישראלים','ישראל').str.replace('שישראלי','ישראל').str.replace('כשישראלי','ישראל')
                           .str.replace('לישראלי','ישראל'))

subset_1['clean_new'] = (subset_1['clean_new'].str.replace('שערים','שער').str.replace('השער','שער')
                            .str.replace('לשער','שער').str.replace('השערים','שער').str.replace('משערי','שער')
                            .str.replace('בשער','שער').str.replace('שלושער','שער').str.replace('משער','שער')
                           .str.replace('מהשער','שער').str.replace('שערי','שער').str.replace('ששער','שער')
                           .str.replace('ושער','שער'))

subset_1['clean_new'] = (subset_1['clean_new'].str.replace('הקבוצה','קבוצה').str.replace('בקבוצה','קבוצה')
                            .str.replace('לקבוצה','קבוצה').str.replace('שהקבוצה','קבוצה').str.replace('וקבוצה','קבוצה'))


subset_1['clean_new'] = (subset_1['clean_new'].str.replace('דקות','דקה').str.replace('בדקה','דקה')
                            .str.replace('מדקה','דקה').str.replace('בדקות','דקה').str.replace('לדקה','דקה')
                            .str.replace('מהדקות','דקה'))

subset_1['clean_new'] = (subset_1['clean_new'].str.replace('בעונה','עונה').str.replace('לעונה','עונה')
                            .str.replace('מהעונה','עונה').str.replace('מעונה','עונה').str.replace('לדקה','דקה')
                            .str.replace('מהדקות','דקה'))

subset_1['clean_new'] = (subset_1['clean_new'].str.replace('במכבי','מכבי').str.replace('למכבי','מכבי')
                            .str.replace('ממכבי','מכבי').str.replace('ומכבי','מכבי').str.replace('שמכבי','מכבי')
                            .str.replace('מהדקות','דקה'))

subset_1['clean_new'] = (subset_1['clean_new'].str.replace('במכבי','מכבי').str.replace('למכבי','מכבי')
                            .str.replace('ממכבי','מכבי').str.replace('ומכבי','מכבי').str.replace('שמכבי','מכבי'))

subset_1['clean_new'] = (subset_1['clean_new'].str.replace('במקום','מקום').str.replace('למקום','מקום')
                            .str.replace('מקומות','מקום').str.replace('ובמקום','מקום').str.replace('מקומו','מקום')
                            .str.replace('במקומם','מקום').str.replace('ממקום','מקום').str.replace('במקומות','מקום'))

subset_1['clean_new'] = (subset_1['clean_new'].str.replace('במקום','מקום').str.replace('למקום','מקום')
                            .str.replace('מקומות','מקום').str.replace('ובמקום','מקום').str.replace('מקומו','מקום')
                            .str.replace('במקומם','מקום').str.replace('ממקום','מקום').str.replace('במקומות','מקום'))

subset_1['clean_new'] = (subset_1['clean_new'].str.replace('הליגה','בליגה').str.replace('לליגה','בליגה')
                            .str.replace('ליגה','בליגה').str.replace('הליגות','בליגה'))

subset_1['clean_new'] = (subset_1['clean_new'].str.replace('ההרכב','הרכב').str.replace('בהרכב','הרכב')
                            .str.replace('ההרכבים','הרכב').str.replace('הרכבים','הרכב').str.replace('להרכב','הרכב'))

subset_1['clean_new'] = (subset_1['clean_new'].str.replace('יהודי','יהודים').str.replace('יהודי','יהודים')
                            .str.replace('היהודים','יהודים').str.replace('היהודי','יהודים').str.replace('שיהודיה','יהודים')
                            .str.replace('ליהודים','יהודים').str.replace('מהיהודים','יהודים').str.replace('יהודית','יהודים')
                           .str.replace('יהודיות','יהודים'))

subset_1['clean_new'] = (subset_1['clean_new'].str.replace('המנון','ההמנון').str.replace('ההימנון','ההמנון')
                            .str.replace('בהמנון','ההמנון').str.replace('בהימנון','ההמנון').str.replace('המנונים','ההמנון')
                            .str.replace('שבהמנון','ההמנון').str.replace('שבהימנון','ההמנון').str.replace('להמנון','ההמנון')
                           .str.replace('להימנון','ההמנון'))

subset_1['clean_new'] = (subset_1['clean_new'].str.replace('מוסלמים','מוסלמי').str.replace('המוסלמים','מוסלמי')
                            .str.replace('ממוסלמי','מוסלמי').str.replace('למוסלמי','מוסלמי').str.replace('למוסלמים','מוסלמי')
                            .str.replace('שמוסלמי','מוסלמי').str.replace('שמוסלמים','מוסלמי'))


subset_1['clean_new'] = (subset_1['clean_new'].str.replace('גזענית','גזעני').str.replace('גזענות','גזעני')
                            .str.replace('הגזענית','גזעני').str.replace('גזען','גזעני').str.replace('הגזען','גזעני')
                            .str.replace('גזעניים','גזעני').str.replace('הגזענות','גזעני').str.replace('מהגזענים','גזעני')
                           .str.replace('גזענים','גזעני').str.replace('הגזעניים','גזעני').str.replace('גזעי','גזעני')
                           .str.replace('וגזעי','גזעני').str.replace('גיזענית','גזעני').str.replace('גיזעני','גזעני')
                           .str.replace('הגיזענית','גזעני').str.replace('גיזען','גזעני').str.replace('גיזעני','גזעני')
                           .str.replace('גיזעניים','גזעני').str.replace('הגיזענות','גזעני').str.replace('מהגיזענים','גזעני')
                           .str.replace('גיזענים','גזעני').str.replace('הגיזעניים','גזעני').str.replace('גיזענות','גזעני'))


In [5]:
# prepare the data and create train and test sets
def get_feature_vector(train_fit):
    vector = TfidfVectorizer(sublinear_tf=True)
    vector.fit(train_fit)
    return vector

tf_vector = get_feature_vector(np.array(subset_1.iloc[:, 5].values.astype('U')).ravel())
X = tf_vector.transform(np.array(subset_1.iloc[:, 5].values.astype('U')).ravel())
y = np.array(subset_1.iloc[:, 3]).ravel()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# create folds for cross validation
k = 10
kf_10 = KFold(n_splits = k, random_state = 0, shuffle=True)

In [6]:
# Logisitic regression

lm_accuracy_test_list = []
lm_f1_test_list = []

LM_model = LogisticRegressionCV(penalty='l2', max_iter=500, multi_class='multinomial', solver='lbfgs',
                                cv=10)
LM_model.fit(X_train,y_train)

#Target prediction & F1 score from test set.
y_pred_test = LM_model.predict(X_test)
lm_accuracy_test = accuracy_score(y_test, y_pred_test)
lm_accuracy_test_list.append(lm_accuracy_test)
lm_f1_test = f1_score(y_test, y_pred_test, average='weighted')
lm_f1_test_list.append(lm_f1_test)

#print(lm_accuracy_test_list)
#print(lm_f1_test_list)

In [7]:
# naive Bayes

nb_accuracy_CV_list = []
nb_f1_CV_list = []
nb_accuracy_test_list = []
nb_f1_test_list = []


# Training Naive Bayes model
NB_model = MultinomialNB()


# cross validation
for train_index, test_index in kf_10.split(X_train):
    X_train_CV, X_test_CV = X[train_index], X[test_index]
    y_train_CV, y_test_CV = y[train_index], y[test_index]
    NB_model.fit(X_train_CV, y_train_CV)

    #Target prediction, accuracy & F1 score from CV.
    y_pred_CV = NB_model.predict(X_test_CV)
    nb_accuracy_CV = accuracy_score(y_test_CV, y_pred_CV)
    nb_accuracy_CV_list.append(nb_accuracy_CV)
    nb_f1_CV = f1_score(y_test_CV, y_pred_CV, average = 'weighted')
    nb_f1_CV_list.append(nb_f1_CV)

    #Target prediction & F1 score  from test set.
    y_pred_test = NB_model.predict(X_test)
    nb_accuracy_test = accuracy_score(y_test, y_pred_test)
    nb_accuracy_test_list.append(nb_accuracy_test)
    nb_f1_test = f1_score(y_test, y_pred_test, average='weighted')
    nb_f1_test_list.append(nb_f1_test)
    

#print(np.mean(nb_accuracy_test_list))
#print(np.mean(nb_f1_test_list))

In [8]:
# random forest

max_depth = [5, 8, 12, 15, 20]
max_leaf_nodes = [15, 18, 22, 25, 30]


rfc_accuracy_CV_list = []
rfc_f1_CV_list = []
rfc_accuracy_test_list = []
rfc_f1_test_list = []

for i in max_depth:
    for j in max_leaf_nodes:
        # random forest model
        rf_model = RandomForestClassifier(max_depth=i, max_leaf_nodes = j, random_state=0)
        
        # cross validation
        for train_index, test_index in kf_10.split(X_train):
            X_train_CV, X_test_CV = X[train_index], X[test_index]
            y_train_CV, y_test_CV = y[train_index], y[test_index]
            rf_model.fit(X_train_CV, y_train_CV)

            #Target prediction, accuracy & F1 score from CV.
            y_pred_CV = rf_model.predict(X_test_CV)
            rfc_accuracy_CV = accuracy_score(y_test_CV, y_pred_CV)
            rfc_accuracy_CV_list.append(rfc_accuracy_CV)
            rfc_f1_CV = f1_score(y_test_CV, y_pred_CV, average = 'weighted')
            rfc_f1_CV_list.append([rfc_f1_CV,i,j])

            #Target prediction & F1 score  from test set.
            y_pred_test = rf_model.predict(X_test)
            rfc_accuracy_test = accuracy_score(y_test, y_pred_test)
            rfc_accuracy_test_list.append(rfc_accuracy_test)
            rfc_f1_test = f1_score(y_test, y_pred_test, average='weighted')
            rfc_f1_test_list.append([rfc_f1_test,i,j])
            

# print(sorted(rfc_accuracy_test_list)[-1::-1])
# print(sorted(rfc_f1_test_list)[-1::-1])

In [9]:
# SVM - poly

d = [0.5,1,1.5,2,2.5]
C = [0.3,0.6,0.9,1.2,1.5]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

poly_accuracy_CV_list = []
poly_f1_CV_list = []
poly_accuracy_test_list = []
poly_f1_test_list = []

for i in d:
    for j in C:
        # svm poly model
        model_poly = svm.SVC(kernel='poly', degree=i, C=j)

        # cross validation
    
        for train_index, test_index in kf_10.split(X_train):
            X_train_CV, X_test_CV = X[train_index], X[test_index]
            y_train_CV, y_test_CV = y[train_index], y[test_index]
            model_poly.fit(X_train_CV, y_train_CV)

            #Target prediction, accuracy & F1 score from CV.
            y_pred_CV = model_poly.predict(X_test_CV)
            poly_accuracy_CV = accuracy_score(y_test_CV, y_pred_CV)
            poly_accuracy_CV_list.append(poly_accuracy_CV)
            poly_f1_CV = f1_score(y_test_CV, y_pred_CV, average = 'weighted')
            poly_f1_CV_list.append([poly_f1_CV,i,j])

            #Target prediction & F1 score  from test set.
            y_pred_test = model_poly.predict(X_test)
            poly_accuracy_test = accuracy_score(y_test, y_pred_test)
            poly_accuracy_test_list.append(poly_accuracy_test)
            poly_f1_test = f1_score(y_test, y_pred_test, average='weighted')
            poly_f1_test_list.append([poly_f1_test,i,j])

            
# print(sorted(poly_accuracy_test_list)[-1::-1])
# print(sorted(poly_f1_test_list)[-1::-1])

In [11]:
# SVM - rbf

degree = [0.5,1,1.5,2,2.5]
C = [0.3,0.6,0.9,1.2,1.5]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

rbf_accuracy_CV_list = []
rbf_f1_CV_list = []
rbf_accuracy_test_list = []
rbf_f1_test_list = []

for i in d:
    for j in C:
        # svm rbf model
        model_rbf = svm.SVC(kernel='rbf', gamma=i, C=j)

        # cross validation
        for train_index, test_index in kf_10.split(X_train):
            X_train_CV, X_test_CV = X[train_index], X[test_index]
            y_train_CV, y_test_CV = y[train_index], y[test_index]
            model_rbf.fit(X_train_CV, y_train_CV)

            #Target prediction, accuracy & F1 score from CV.
            y_pred_CV = model_rbf.predict(X_test_CV)
            rbf_accuracy_CV = accuracy_score(y_test_CV, y_pred_CV)
            rbf_accuracy_CV_list.append(rbf_accuracy_CV)
            rbf_f1_CV = f1_score(y_test_CV, y_pred_CV, average = 'weighted')
            rbf_f1_CV_list.append([rbf_f1_CV,i,j])

            #Target prediction & F1 score  from test set.
            y_pred_test = model_rbf.predict(X_test)
            rbf_accuracy_test = accuracy_score(y_test, y_pred_test)
            rbf_accuracy_test_list.append(rbf_accuracy_test)
            rbf_f1_test = f1_score(y_test, y_pred_test, average='weighted')
            rbf_f1_test_list.append([rbf_f1_test,i,j])

            
# print(sorted(rbf_accuracy_test_list)[-1::-1])
# print(sorted(rbf_f1_test_list)[-1::-1])